In [1]:
# Import dependencies
import os
from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-04-19 16:31:49.963789: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 16:31:49.971169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745073109.979297   19738 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745073109.981729   19738 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745073109.988524   19738 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 19 intra-op, 12 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1745073111.230979   19738 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping
from src.model.tensorflow_utils import start_or_resume_training
from src.model.densnet.tensorflow_dense_net_basic import build_densenet

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "SimplifiedDenseNetV1"
model_name      = f"{set_name}_1"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1745073111.251658   19738 cuda_executor.cc:479] failed to allocate 18.00PiB (20266198323167232 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745073111.251681   19738 cuda_executor.cc:479] failed to allocate 16.20PiB (18239577202360320 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745073111.251684   19738 cuda_executor.cc:479] failed to allocate 14.58PiB (16415619482124288 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745073111.251687   19738 cuda_executor.cc:479] failed to allocate 13.12PiB (14774057211789312 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745073111.251688   19738 cuda_executor.cc:479] failed to allocate 11.81PiB (13296650738991104 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745073111.251690   19738 cuda_executor.cc:

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)
val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

2025-04-19 16:32:01.778167: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 1048576
2025-04-19 16:32:34.918944: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
2025-04-19 16:32:38.529645: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 71842 samples [00:36, 1952.84 samples/s]
2025-04-19 16:32:47.068357: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 15431 samples [00:08, 1807.41 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryCrossentropy(from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

In [5]:
# Model Training
model          = build_densenet(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-19 16:32:47 - INFO - 
=== Training Started ===

2025-04-19 16:32:47 - INFO - Batch Size: 128
2025-04-19 16:32:47 - INFO - Optimizer: Adam
2025-04-19 16:32:47 - INFO - 

2025-04-19 16:32:47 - INFO - 
=== Starting Epoch 1 ===



Epoch 1/100


I0000 00:00:1745073172.512405   19799 service.cc:152] XLA service 0x70f9a40022a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745073172.512425   19799 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1745073172.515768   19799 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745073172.556673   19799 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-19 16:33:08.720794: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 14231 of 25000
2025-04-19 16:33:18.058644: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-04-19 16:33:18.825133: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to 

309/561 ━━━━━━━━━━━━━━━━━━━━ 49s 198ms/step - accuracy: 0.9093 - auc: 0.5597 - f1_score: 0.2375 - loss: 0.8057 - precision: 0.4774 - recall: 0.2282 

2025-04-19 16:35:21.275816: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


...

561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9885 - auc: 0.9869 - f1_score: 0.9221 - loss: 0.0784 - precision: 0.9639 - recall: 0.8993 

2025-04-19 18:38:04 - INFO - 
=== Epoch 48 Summary ===
2025-04-19 18:38:04 - INFO - Time: 152.16s
2025-04-19 18:38:04 - INFO - Training   - accuracy: 0.9888 - auc: 0.9880 - f1_score: 0.9239 - loss: 0.0778 - precision: 0.9645 - recall: 0.9019 - learning_rate: 0.0000
2025-04-19 18:38:04 - INFO - Validation - accuracy: 0.9113 - auc: 0.6561 - f1_score: 0.3949 - loss: 0.4869 - precision: 0.4743 - recall: 0.3778
2025-04-19 18:38:04 - INFO - ==================================================



561/561 ━━━━━━━━━━━━━━━━━━━━ 152s 271ms/step - accuracy: 0.9885 - auc: 0.9869 - f1_score: 0.9221 - loss: 0.0784 - precision: 0.9639 - recall: 0.8993 - val_accuracy: 0.9113 - val_auc: 0.6561 - val_f1_score: 0.3949 - val_loss: 0.4869 - val_precision: 0.4743 - val_recall: 0.3778 - learning_rate: 1.0000e-06


2025-04-19 18:38:04 - INFO - 
=== Training Completed! ===

2025-04-19 18:38:04 - INFO - Final Metrics: accuracy: 0.9888 - auc: 0.9880 - f1_score: 0.9239 - loss: 0.0778 - precision: 0.9645 - recall: 0.9019 - val_accuracy: 0.9113 - val_auc: 0.6561 - val_f1_score: 0.3949 - val_loss: 0.4869 - val_precision: 0.4743 - val_recall: 0.3778



Epoch 48: early stopping
Restoring model weights from the end of the best epoch: 33.
